# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

In [2]:
from azureml.core import Workspace, Experiment
from azureml.pipeline.steps import PythonScriptStep
from azureml.train.sklearn import SKLearn

ws = Workspace.get(name="quick-starts-ws-127563")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster

# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
vm_size = "Standard_DS3_V2"
compute_name="notebook127563"
# create the cluster
compute_min_nodes=0
compute_max_nodes=4
 



### YOUR CODE HERE ###
# Checking if  compute already exist or not  if not creating a new target
if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print("found compute target: " + compute_name)
else:
    print("creating new compute target...")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, timeout_in_minutes=20)
    # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())





Workspace name: quick-starts-ws-127563
Azure region: southcentralus
Subscription id: b4a122b5-b4d5-40e7-9878-57b87adf4a8b
Resource group: aml-quickstarts-127563


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
#ws = Workspace.from_config()

# choose a name for experiment
#experiment_name = 'your experiment name here'

#experiment=Experiment(ws, experiment_name)

#workspace = Workspace(subscription_id, resource_group, workspace_name)

# get the datastore to upload prepared data
datastore = ws.get_default_datastore()

# upload the local file from src_dir to the target_path in datastore
datastore.upload(src_dir='./', target_path='data')

# create a dataset referencing the cloud location
#

Uploading an estimated of 9 files
Target already exists. Skipping upload for data/a.txt
Target already exists. Skipping upload for data/Auto_ML_version_4.ipynb
Target already exists. Skipping upload for data/heart_failure_clinical_records_dataset.csv
Target already exists. Skipping upload for data/.ipynb_checkpoints/Auto_ML_version_4-checkpoint.ipynb
Uploading ./automl.log
Uploaded ./automl.log, 1 files out of an estimated total of 8
Uploading ./automl_best_model.pkl
Uploaded ./automl_best_model.pkl, 2 files out of an estimated total of 8
Uploading ./azureml_automl.log
Uploaded ./azureml_automl.log, 3 files out of an estimated total of 7
Uploading ./score.py
Uploaded ./score.py, 4 files out of an estimated total of 6
Uploading ./.ipynb_checkpoints/score-checkpoint.py
Uploaded ./.ipynb_checkpoints/score-checkpoint.py, 5 files out of an estimated total of 5
Uploaded 5 files


$AZUREML_DATAREFERENCE_574974c3e11e4ed5b10bbb8cb93c63d7

In [4]:
dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, ('data/heart_failure_clinical_records_dataset.csv'))])

In [5]:
x_df=dataset.to_pandas_dataframe()

In [ ]:
"""
est = SKLearn(source_directory="./", 
                compute_target=compute_target, 
                vm_size=vm_size,
                entry_script='train.py'
               )

"""

In [6]:
dataset

{
  "source": [
    "('workspaceblobstore', 'data/heart_failure_clinical_records_dataset.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
# TODO: Put your automl settings here



from azureml.train.automl import AutoMLConfig
import numpy as np


# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    compute_target=compute_target,
    primary_metric="accuracy",
    training_data=dataset,
    label_column_name='DEATH_EVENT',
    #allowed_models=["LightGBM","RandomForest"],
    n_cross_validations=3)


In [16]:
exp=Experiment(ws,"automl_test")
automl_model=exp.submit(automl_config,show_output=True)
### YOUR CODE HERE ###
import joblib
from azureml.widgets import RunDetails
RunDetails(automl_model).show()
best_run, fitted_model = automl_model.get_output()
print(best_run)
print(fitted_model)
from azureml.core.model import Model
import joblib
joblib.dump(fitted_model,"automl_best_model.pkl")


Running on remote.
Running on remote compute: notebook127563
Parent Run ID: AutoML_4f78626b-04e8-45bf-8e6b-9bae91cd85c2

Received interrupt. Returning now.

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

AutoMLException: AutoMLException:
	Message: Could not find a model with valid score for metric 'accuracy'. Please ensure that at least one run was successfully completed with a valid score for the given metric.
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Could not find a model with valid score for metric 'accuracy'. Please ensure that at least one run was successfully completed with a valid score for the given metric.",
        "target": "metric",
        "inner_error": {
            "code": "NotFound",
            "inner_error": {
                "code": "ModelMissing"
            }
        }
    }
}

In [7]:
from azureml.core.model import Model
import joblib
model = Model.register(workspace=ws, model_path="automl_best_model.pkl", model_name="test")

Registering model test


In [8]:
from azureml.core.webservice import LocalWebservice
from azureml.core.model import InferenceConfig
local_config = LocalWebservice.deploy_configuration(port=9000)
inference_config = InferenceConfig(entry_script='score.py', environment=best_run.get_environment())

local_service = Model.deploy(ws, "test", [model], inference_config, local_config)

local_service.wait_for_deployment(show_output=True)

NameError: name 'best_run' is not defined

In [30]:

x_df=dataset.to_pandas_dataframe()

KeyboardInterrupt: 

In [29]:


#input_data=x_df[0:2].to_dict(orient='records')
import json
#i=json.dumps(input_data)
local_service.run(i)

TypeError: 'NoneType' object is not subscriptable

In [28]:
i

'{"data": [{"age": 75.0, "anaemia": 0, "creatinine_phosphokinase": 582, "diabetes": 0, "ejection_fraction": 20, "high_blood_pressure": 1, "platelets": 265000.0, "serum_creatinine": 1.9, "serum_sodium": 130, "sex": 1, "smoking": 0, "time": 4}]}'

In [15]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice


service_name = 'custom-env-service'

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [13]:
service.get_logs()

'2020-11-23T04:05:08.2373015Z stdout F 2020-11-23T04:05:08,226298800+00:00 - gunicorn/run \n2020-11-23T04:05:08.2543037Z stdout F 2020-11-23T04:05:08,243031400+00:00 - iot-server/run \n2020-11-23T04:05:08.2653018Z stdout F 2020-11-23T04:05:08,254408900+00:00 - rsyslog/run \n2020-11-23T04:05:08.2942928Z stdout F 2020-11-23T04:05:08,283487500+00:00 - nginx/run \n2020-11-23T04:05:08.3013311Z stderr F /usr/sbin/nginx: /azureml-envs/azureml_0e3a8a6dba181476a2523c12c58dfc97/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n2020-11-23T04:05:08.3013311Z stderr F /usr/sbin/nginx: /azureml-envs/azureml_0e3a8a6dba181476a2523c12c58dfc97/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n2020-11-23T04:05:08.3053191Z stderr F /usr/sbin/nginx: /azureml-envs/azureml_0e3a8a6dba181476a2523c12c58dfc97/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n2020-11-23T04:05:08.3053191Z stderr F /usr/sbi

KeyboardInterrupt: 

In [47]:
service.scoring_uri

'http://e5401dfb-1add-43be-987a-87e391a83eb3.southcentralus.azurecontainer.io/score'

In [18]:
input_data={"data":[{'age': 75.0,
  'anaemia': 0,
  'creatinine_phosphokinase': 582,
  'diabetes': 0,
  'ejection_fraction': 20,
  'high_blood_pressure': 1,
  'platelets': 265000.0,
  'serum_creatinine': 1.9,
  'serum_sodium': 130,
  'sex': 1,
  'smoking': 0,
  'time': 4}]}
import json
i=json.dumps(input_data)

In [21]:
service.update(auth_enabled=True)

In [64]:
with open("data.json","w") as _f:
    _f.write(i)
    

In [33]:
headers={"Content-Type":"application/json"}
key="nmzf01o9jlalITeEPXejCGkw7Bqs0Iv0"
headers["Authorization"]=f'Bearer "hL84fRidVsQHrZsgj9he798pG7tEyr6c"'

In [29]:
!az login

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RWJDCH8LX to authenticate.
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "660b3398-b80e-49d2-bc5b-ac1dc93b5254",
    "id": "dafd62fe-51d3-4671-af50-6bac2f001b5d",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Azure Sponsorship - Udacity -47",
    "state": "Enabled",
    "tenantId": "660b3398-b80e-49d2-bc5b-ac1dc93b5254",
    "user": {
      "name": "odl_user_127468@udacitylabs.onmicrosoft.com",
      "type": "user"
    }
  }
]


In [34]:
import requests
result=requests.post('http://b16e4ef9-3001-4d9c-82f2-b0f1c7850e7d.southcentralus.azurecontainer.io/score',i,headers=headers)

In [36]:
result.text


'Unauthorized, no token matched'

In [ ]:

service.delete()